In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [ ]:
size = 1000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
    pds.random(0., 1.).alias("x2"),
    pds.random(0., 1.).alias("x3"),
).with_columns(
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x1")).alias("x1"),
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x2")).alias("x2"),
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x3")).alias("x3"),
).with_columns(
    null_ref = pl.any_horizontal(pl.col("x1").is_null(), pl.col("x2").is_null(), pl.col("x3").is_null()),
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)

window_size = 6
min_valid_rows = 5

result = df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "exp(x2)", "x3",
        target = "y",
        window_size = window_size,
        min_valid_rows = min_valid_rows,
        null_policy = "skip" 
    ).alias("test")
).with_columns(
    pl.col("test").is_null().alias("is_null")
)